In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from monitor import interact
import gym
import numpy as np

In [19]:
from collections import defaultdict

class Agent:

    def __init__(self, nA=6, eps=0.005, alpha=1, gamma=1.0):
        """ Initialize agent.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA
        self.Q = defaultdict(lambda: np.zeros(self.nA))
        self.eps = eps
        
        self.alpha = alpha
        self.gamma = gamma
        
    def select_action(self, state):
        """ Given the state, select an action.

        Params
        ======
        - state: the current state of the environment

        Returns
        =======
        - action: an integer, compatible with the task's action space
        """
        if np.random.random() > self.eps:
            return np.argmax(self.Q[state])
        else:
            return np.random.choice(self.nA)

    def step(self, state, action, reward, next_state, done):
        """ Update the agent's knowledge, using the most recently sampled tuple.

        Params
        ======
        - state: the previous state of the environment
        - action: the agent's previous choice of action
        - reward: last reward received
        - next_state: the current state of the environment
        - done: whether the episode is complete (True or False)
        """
        current = self.Q[state][action]
        greedy_action = np.argmax(self.Q[next_state])
        
        policy_probs = np.ones(self.nA) * (self.eps / self.nA)
        policy_probs[greedy_action] += (1-self.eps)
        
        expected_Q = sum(policy_probs * self.Q[next_state])
        
        self.Q[state][action] = current + (self.alpha * (reward + (self.gamma * expected_Q) - current))

In [4]:
env = gym.make('Taxi-v3')

In [5]:
# https://wooono.tistory.com/102

from bayes_opt import BayesianOptimization

# Set of hyperparemeters
pbounds = {'eps': (0.001, 0.1),
            'alpha': (0.1, 1),
            'gamma': (0.3, 1),
            }

# Function to optimize
def opt_function(eps, alpha, gamma):
    agent = Agent(eps=eps, alpha=alpha, gamma=gamma)
    _, best_avg_reward = interact(env, agent, verbose=False)
    
    return best_avg_reward

# Bayesian optimization Object
# f : Target Function, pbounds : hyperparameter set
# verbose = 2 Print always, verbose = 1 Print when find maximum, verbose = 0 No Print
# random_state : Seed
bo=BayesianOptimization(f=opt_function, pbounds=pbounds, verbose=2, random_state=47)    

# Run target value maximize
# init_points :  Initial Random Search Counts
# n_iter : The number of Iteration
# acq : Acquisition Functions - Expected Improvement(EI), Probability of Improvement(PI), Upper Confidence Bound(UCB)
# xi : Exploration Strength
bo.maximize(init_points=3, n_iter=50, acq='ei', xi=0.01)

print(bo.max)

|   iter    |  target   |   alpha   |    esp    |   gamma   |
-------------------------------------------------------------
|  1        |  4.82     |  0.2021   |  0.09747  |  0.8101   |
|  2        |  6.15     |  0.4163   |  0.07105  |  0.8597   |
|  3        |  7.27     |  0.681    |  0.04205  |  0.7942   |
|  4        |  6.53     |  0.9757   |  0.05574  |  0.9547   |
|  5        |  5.0      |  0.786    |  0.08894  |  0.9198   |
|  6        |  4.68     |  0.3885   |  0.08556  |  0.6104   |
|  7        |  6.98     |  0.9826   |  0.05415  |  0.8778   |
|  8        |  7.75     |  0.6564   |  0.03034  |  0.7665   |
|  9        |  8.92     |  0.6655   |  0.001    |  0.7151   |
|  10       |  8.82     |  0.6968   |  0.001    |  0.6649   |
|  11       |  8.78     |  0.6416   |  0.001    |  0.6706   |
|  12       |  5.12     |  0.22     |  0.08105  |  0.6873   |
|  13       |  8.96     |  0.9369   |  0.001    |  0.6219   |
|  14       |  8.45     |  0.8684   |  0.006579 |  0.4133   |
|  15   

In [15]:
from collections import defaultdict

class Agent2:

    def __init__(self, nA=6, eps=0.005, eps_decay=0.9, alpha=1, gamma=1.0):
        """ Initialize agent.

        Params
        ======
        - nA: number of actions available to the agent
        """
        self.nA = nA
        self.Q = defaultdict(lambda: np.zeros(self.nA))
        self.eps = eps
        self.eps_decay = eps_decay
        
        self.alpha = alpha
        self.gamma = gamma
        
    def get_policy_probs(self, state):
        greedy_action = np.argmax(self.Q[state])        
        policy_probs = np.ones(self.nA) * (self.eps / self.nA)
        policy_probs[greedy_action] += (1-self.eps)
        
        return policy_probs
        
    def select_action(self, state):
        """ Given the state, select an action.

        Params
        ======
        - state: the current state of the environment

        Returns
        =======
        - action: an integer, compatible with the task's action space
        """
        return np.random.choice(self.nA, p=self.get_policy_probs(state))

    def step(self, state, action, reward, next_state, done):
        """ Update the agent's knowledge, using the most recently sampled tuple.

        Params
        ======
        - state: the previous state of the environment
        - action: the agent's previous choice of action
        - reward: last reward received
        - next_state: the current state of the environment
        - done: whether the episode is complete (True or False)
        """
        current = self.Q[state][action]
        policy_probs = self.get_policy_probs(next_state)
        expected_Q = sum(policy_probs * self.Q[next_state])
        
        self.Q[state][action] = current + (self.alpha * (reward + (self.gamma * expected_Q) - current))
        
        if done:
            self.eps = self.eps * self.eps_decay

In [17]:
pbounds2 = {'eps': (0.001, 0.1),
            'eps_decay': (0.1, 0.9),
            'alpha': (0.1, 1),
            'gamma': (0.3, 1),
            }

def opt_function2(eps, eps_decay, alpha, gamma):
    agent = Agent2(eps=eps, eps_decay=eps_decay, alpha=alpha, gamma=gamma)
    _, best_avg_reward = interact(env, agent, verbose=False)
    
    return best_avg_reward

bo2 = BayesianOptimization(f=opt_function2, pbounds=pbounds2, verbose=2, random_state=47)    
bo2.maximize(init_points=3, n_iter=30, acq='ei', xi=0.01)
print(bo2.max)

|   iter    |  target   |   alpha   |    eps    | eps_decay |   gamma   |
-------------------------------------------------------------------------
|  1        |  9.11     |  0.2021   |  0.09747  |  0.683    |  0.546    |
|  2        |  8.54     |  0.7368   |  0.08016  |  0.6164   |  0.5902   |
|  3        |  8.73     |  0.7354   |  0.02542  |  0.3048   |  0.3168   |
|  4        |  9.0      |  0.215    |  0.09028  |  0.6996   |  0.5469   |
|  5        |  8.73     |  0.2044   |  0.09766  |  0.6244   |  0.5422   |
|  6        |  8.83     |  0.2266   |  0.07713  |  0.7246   |  0.5572   |
|  7        |  8.7      |  0.1849   |  0.1      |  0.6913   |  0.5427   |
|  8        |  8.89     |  0.7373   |  0.08983  |  0.8958   |  0.7685   |
|  9        |  8.7      |  0.4025   |  0.01256  |  0.3243   |  0.6511   |
|  10       |  8.9      |  0.9292   |  0.03196  |  0.1758   |  0.3413   |
|  11       |  8.85     |  0.5644   |  0.01335  |  0.3704   |  0.6178   |
|  12       |  8.78     |  0.831    | 

In [22]:
bo3=BayesianOptimization(f=opt_function, pbounds=pbounds, verbose=2, random_state=47)    
bo3.maximize(init_points=3, n_iter=30)
print(bo3.max)

|   iter    |  target   |   alpha   |    eps    |   gamma   |
-------------------------------------------------------------
|  1        |  5.74     |  0.2021   |  0.09747  |  0.8101   |
|  2        |  6.36     |  0.4163   |  0.07105  |  0.8597   |
|  3        |  7.17     |  0.681    |  0.04205  |  0.7942   |
|  4        |  8.19     |  0.6665   |  0.01628  |  0.7712   |
|  5        |  8.38     |  0.6456   |  0.01384  |  0.7457   |
|  6        |  8.71     |  0.7053   |  0.001    |  0.7016   |
|  7        |  9.12     |  0.6711   |  0.001    |  0.5987   |
|  8        |  8.58     |  0.7774   |  0.001    |  0.5595   |
|  9        |  8.8      |  0.6373   |  0.001    |  0.4742   |
|  10       | -3.8      |  0.6725   |  0.1      |  0.5374   |
|  11       | -109.2    |  0.8627   |  0.09445  |  0.398    |
|  12       |  8.55     |  0.667    |  0.004676 |  0.6026   |
|  13       |  8.54     |  0.2931   |  0.001    |  0.4265   |
|  14       |  8.53     |  0.8684   |  0.006579 |  0.4133   |
|  15   

In [23]:
bo4 = BayesianOptimization(f=opt_function2, pbounds=pbounds2, verbose=2, random_state=47)    
bo4.maximize(init_points=3, n_iter=30)
print(bo4.max)

|   iter    |  target   |   alpha   |    eps    | eps_decay |   gamma   |
-------------------------------------------------------------------------
|  1        |  8.93     |  0.2021   |  0.09747  |  0.683    |  0.546    |
|  2        |  8.73     |  0.7368   |  0.08016  |  0.6164   |  0.5902   |
|  3        |  8.74     |  0.7354   |  0.02542  |  0.3048   |  0.3168   |
|  4        |  8.99     |  0.1997   |  0.07971  |  0.6642   |  0.5726   |
|  5        |  8.83     |  0.2069   |  0.09399  |  0.6782   |  0.5194   |
|  6        |  8.73     |  0.1781   |  0.06616  |  0.8858   |  0.9891   |
|  7        |  8.84     |  0.1914   |  0.1      |  0.6936   |  0.606    |
|  8        |  8.78     |  0.2213   |  0.08022  |  0.619    |  0.5732   |
|  9        |  8.73     |  0.1918   |  0.08865  |  0.6577   |  0.5579   |
|  10       |  8.75     |  0.9292   |  0.03196  |  0.1758   |  0.3413   |
|  11       |  8.89     |  0.5644   |  0.01335  |  0.3704   |  0.6178   |
|  12       |  8.85     |  0.831    | 